# PyTorch To ONNX Model Converter Google Colab (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# NVIDIA GPU Check

In [ ]:
!nvidia-smi

# Setup Environment

## Run cells below one at a time and make sure to restart the colab runtime/re-run each cell again after the run to make sure that there are no residual install errors or conflicts

In [ ]:
!pip uninstall torch

In [ ]:
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

In [ ]:
!pip install numpy -U

In [ ]:
!pip install transformers -U
!pip install onnx -U
!pip install onnxruntime -U
!pip install onnxruntime-gpu -U
!pip install einops -U
!pip install huggingface_hub -U

# Prep and Load Your (Custom) PyTorch Model/Architecture

## Below we are going to use Allegro Music Transformer PyTorch Music AI model/custom architecture as an example

# Download the PyTorch model

In [ ]:
from huggingface_hub import hf_hub_download

hf_hub_download(repo_id='asigalov61/Allegro-Music-Transformer',
                filename='Allegro_Music_Transformer_Small_Trained_Model_56000_steps_0.9399_loss_0.7374_acc.pth',
                local_dir='/content/',
                local_dir_use_symlinks=False)

# Prep custom PyTorch model environment/architecture

In [ ]:
!git clone https://github.com/asigalov61/tegridy-tools

%cd /content/tegridy-tools/tegridy-tools

import TMIDIX

%cd /content/tegridy-tools/tegridy-tools/X-Transformer

from x_transformer import *

%cd /content/

# Load custom PyTorch model

In [ ]:
import torch

device = torch.device('cuda:0')

SEQ_LEN = 2048

# instantiate the model

model = TransformerWrapper(
    num_tokens = 3088,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = 1024, depth = 32, heads = 8)
).cuda()

model = AutoregressiveWrapper(model).cuda() #, mask_prob = 0.15)
model = torch.nn.DataParallel(model).cuda()
model.cuda()
model.load_state_dict(torch.load('/content/Allegro_Music_Transformer_Small_Trained_Model_56000_steps_0.9399_loss_0.7374_acc.pth'))
model.cuda()



model.eval()
model.cuda()

model.to(device=device)
print('Done!')

# Generate seq_len sized output from the PyTorch model to use with PyTorch ONNX exporter. Save it for later use as well.

## Please note that generating seq_len output may take up-to 10 minutes on T4 GPU so please be patient :)

In [ ]:
x = torch.LongTensor([[2816]]).cuda()

torch_out = model.module.generate(x, 2048)

In [ ]:
torch_out[0].tolist()

# Save the generated output from the source model

In [ ]:
with open('torch_out.txt', 'w') as file:
   file.write('\n'.join(str(to) for to in torch_out[0].tolist()))

# Load and prep the generated output from the source model for use with ONNX exporter

In [ ]:
with open("torch_out.txt", 'r') as file:
  t_out = file.readlines()

torch_out1 = []
for t in t_out:
  torch_out1.append(int(t))

In [ ]:
torch_out2 = torch.LongTensor([torch_out1]).to(device=device)

In [ ]:
torch_out2

# Start the ONNX export/conversion to ONNX format!

## You can ignore grey warning messages from the exporter!

In [ ]:
import torch


def export_onnx(mod, model_inputs, path):

    torch.onnx.export(mod,  # model being run
                      model_inputs,  # model input (or a tuple for multiple inputs)
                      path,  # where to save the model (can be a file or file-like object)
                      export_params=True,  # store the trained parameter weights inside the model file
                      opset_version=14,  # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      verbose=True,
                      input_names=['input'],
                      output_names=['output'],
                      dynamic_axes={ # dict value: manually named axes
                                    "input": {1: "seq_len"},
                                    # list value: automatic names
                                    "output": [0],
                                }
                      )

    print('finished exporting onnx')

export_onnx(model.module.net.to(device=device), torch_out2, path='/content/test.bin')

# Test the resulting ONNX model (graph)

In [ ]:
import onnx

# Load the ONNX model
omodel = onnx.load("/content/test.bin")

# Check that the model is well formed
onnx.checker.check_model(omodel)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(omodel.graph))

# Check IO names, shapes, and types

In [ ]:
import onnxruntime as ort

session = ort.InferenceSession('/content/test.bin', None, providers=['CUDAExecutionProvider'])
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print(input_name)
print(output_name)

In [ ]:
input_name = session.get_inputs()[0].name
print("input name", input_name)
input_shape = session.get_inputs()[0].shape
print("input shape", input_shape)
input_type = session.get_inputs()[0].type
print("input type", input_type)

output_name = session.get_outputs()[0].name
print("output name", output_name)
output_shape = session.get_outputs()[0].shape
print("output shape", output_shape)
output_type = session.get_outputs()[0].type
print("output type", output_type)

# Now we are going to test the resulting ONNX model for inference!

# Load the sample model input here to prime the model with

In [ ]:
with open("torch_out.txt", 'r') as file:
  t_out = file.readlines()

torch_out1 = []
for t in t_out:
  torch_out1.append(int(t))

# Initialize the ONNX model!

In [ ]:
import onnxruntime

session = onnxruntime.InferenceSession('/content/test.bin', providers=['CUDAExecutionProvider'])

# Below we are going to init a simple autoregressive wrapper function to eval the model and the output!

In [ ]:
import torch
import torch.nn.functional as F

#====================================================

@torch.no_grad()
def generate(
            start_tokens,
            seq_len,
            max_seq_len = 2048,
            temperature = 0.9,
            verbose=True,
            return_prime=False,
            ):

    out = torch.FloatTensor([start_tokens])

    st = len(start_tokens)

    if verbose:
      print("Generating sequence of max length:", seq_len)

    for s in range(seq_len):
        x = out[:, -max_seq_len:]

        torch_in = x.tolist()[0]

        logits = torch.FloatTensor(session.run(None, {'input': [torch_in]})[0])[:, -1]

        filtered_logits = logits

        probs = F.softmax(filtered_logits / temperature, dim=-1)

        sample = torch.multinomial(probs, 1)

        out = torch.cat((out, sample), dim=-1)

        if verbose:
          if s % 32 == 0:
            print(s, '/', seq_len)

    if return_prime:
      return out[:, :]

    else:
      return out[:, st:]

# Generation time!!!

In [ ]:
melody_chords_f = generate([3087, 3073+1, 3075+1], 512)

# Convert the generated tokens/output to MIDI and enjoy! :)

In [ ]:
import TMIDIX

melody_chords_f = melody_chords_f.tolist()[0]

print('=' * 70)
print('Sample INTs', melody_chords_f[:12])
print('=' * 70)

if len(melody_chords_f) != 0:

    song = melody_chords_f
    song_f = []
    time = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    for sss in song:

      ss = int(sss)

      if ss > 0 and ss < 256:

          time += ss * 8

      if ss >= 256 and ss < 1280:

          dur = ((ss-256) // 8) * 32
          vel = (((ss-256) % 8)+1) * 15

      if ss >= 1280 and ss < 2816:
          channel = (ss-1280) // 128
          pitch = (ss-1280) % 128

          song_f.append(['note', time, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Allegro Music Transformer',
                                                        output_file_name = '/content/Allegro-Music-Transformer-Music-Composition',
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)
    print('=' * 70)

# Congrats! You did it! :)